In [1]:
%env AWS-PROFILE=just-my-links

import boto3
import requests
import json


env: AWS-PROFILE=just-my-links


In [3]:
cloudformation_outputs_res = !aws cloudformation describe-stacks --stack-name just-my-links-dev --query 'Stacks[0].Outputs'
cloudformation_outputs = {x['OutputKey']: x['OutputValue'] for x in json.loads(''.join(cloudformation_outputs_res))}
application_bucket_name = cloudformation_outputs['ApplicationBucketName']
document_store_uri = cloudformation_outputs['DocumentStorageHttpApiUrl']
store_document_arn = cloudformation_outputs['StoreDocumentFunctionArn']
index_documents_arn = cloudformation_outputs['IndexDocumentsFunctionArn']
auth_token_secret_arn = cloudformation_outputs['AuthTokenSecretName']

secrets_client = boto3.client('secretsmanager')
auth_token_response = secrets_client.get_secret_value(SecretId=auth_token_secret_arn)
auth_token = auth_token_response['SecretString']

See the logs

In [9]:
!aws logs tail /aws/lambda/just-my-links--store-document--dev

Traceback (most recent call last)::00 2025/06/07/[$LATEST]b638a0c27d3c4397b337403736f5e4c1 [ERROR] Runtime.ImportModuleError: Unable to import module 'app': cannot import name 'parse_options_header' from 'multipart' (/var/lang/lib/python3.13/site-packages/multipart/__init__.py)
2025-06-07T04:49:50.080000+00:00 2025/06/07/[$LATEST]b638a0c27d3c4397b337403736f5e4c1 INIT_REPORT Init Duration: 551.72 ms	Phase: init	Status: error	Error Type: Runtime.ImportModuleError
Traceback (most recent call last)::00 2025/06/07/[$LATEST]b638a0c27d3c4397b337403736f5e4c1 [ERROR] Runtime.ImportModuleError: Unable to import module 'app': cannot import name 'parse_options_header' from 'multipart' (/var/lang/lib/python3.13/site-packages/multipart/__init__.py)
2025-06-07T04:49:50.567000+00:00 2025/06/07/[$LATEST]b638a0c27d3c4397b337403736f5e4c1 INIT_REPORT Init Duration: 467.27 ms	Phase: invoke	Status: error	Error Type: Runtime.ImportModuleError
2025-06-07T04:49:50.567000+00:00 2025/06/07/[$LATEST]b638a0c27d3c4

In [8]:
import urllib.parse

# Test document storage with multipart form data
document_url = "https://example.com/test-article"
document_content = "<html><body><h1>Test Article</h1><p>This is a test article content.</p></body></html>"

# URL encode the document_url for the path parameter
encoded_url = urllib.parse.quote(document_url, safe='')
endpoint_url = f"{document_store_uri}/document/{encoded_url}"

# Prepare multipart form data
files = {
    'document': ('document.html', document_content, 'text/html')
}

response = requests.put(
    endpoint_url,
    headers={"Authorization": f"Bearer {auth_token}"},
    files=files
)

response.status_code, response.text

(404, '{"statusCode":404,"message":"Not found"}')

Deploy changes to both lambda services

In [ ]:
!scripts/deploy-document-storage-service.py

In [ ]:
!scripts/deploy-indexing-service.py

Turn on debug logging with

In [7]:
function_name = 'just-my-links--store-document--dev'
lambda_client = boto3.client('lambda')

response = lambda_client.get_function_configuration(FunctionName=function_name)

current_variables = {**response.get('Environment', {}).get('Variables', {}), "LOG_LEVEL": "DEBUG"}

lambda_client.update_function_configuration(FunctionName=function_name, Environment={'Variables': current_variables})
"debug on"

'debug on'